In [ ]:
#install with: pip install cplearn

### Load dataset X as you would have in the pre-clustering step.

Type of the dataset should be np.ndarray() of shape (n_cells, n_features) where n_cells is the number of cells and n_features is the number of features (PCA components or Harmony components).




In [ ]:
#Load your matrix here.
#Load the pre-clustering matrix (such as post harmony 30 dimensional dataset) as X
#Load the cell x gene matrix as X_cg
#If you have name of the genes, load them in gene_names.

### Load cplearn modules.
1. CorespectModel: Main class for running the Corespect algorithm.
2. CoreSpectConfig: Configuration class for setting up parameters for Corespect.

In [ ]:
from cplearn.corespect import CorespectModel
from cplearn.corespect.config import CoreSpectConfig

### Primary parameters:

1. #### q,r: default: 20,10.
        Neighborhood parameters for K-NN graph construction and ascending random walk of FlowRank respectively. Larger values detect more globally low-variance regions whereas lower values lead to more locally low-variance regions.

2. #### granularity: default: 1.5.
        Another parameter to control locality of cores. Higher values lead to more local cores.

3. #### core_frac: default: 0.2.
        Fraction of points to be selected in the core. Larger fraction is advised if some small clusters are completely missed in the core. Can be auto-selected with auto_select_core_frac:True (beta feature).

4. #### densify: default: False.
        Densification of different induced subgraph. Choose 'rw' or 'k-nn' to try different methods. This reduces the fragmentation of the cores.

5. #### resolution: default: 1.
        Resolution for clustering the core with Leiden. Higher values lead to more clusters in the core.



In [ ]:
#Initial parameters.
cfg = CoreSpectConfig(
    q=20,
    r=10,
    core_frac=0.2,
    densify=False,
    granularity=1.5,
    resolution=1
).configure()

### Running corespect:

We can run it with the two following modes:

1. Fine_grained: This mode allows finding recursively cleaner cores from the initially obtained stable_core. Recommended to be True.

2. Propagate: This mode allows clustering non-core points (and finding peripheral layers) with a normalized-laplacian-based core-label propagation. Recommended to be True.


In [ ]:
#| output: false
model = CorespectModel(X, **cfg.unpack()).run(fine_grained=True,propagate=True)

In [ ]:
import umap
reducer=umap.UMAP()
X_umap=reducer.fit_transform(X)

In [ ]:
#| output: false
from cplearn.coremap import Coremap
cmap=Coremap(model,global_umap=X_umap,fast_view=True)#,anchor_finding_mode='default')#,anchor_reach=500)

In [ ]:
from cplearn.coremap.vizualizer import visualize_coremap
fig=visualize_coremap(cmap,model.labels_, use_webgl=True)
fig.show()

In [ ]:
from scRNA_seq.find_markers import DEAnalyzer,DEOptions

model.count_mat=X_cg

analyzer = DEAnalyzer(model,genes=gene_names,
                       options=DEOptions(pseudocount=1.0, min_cells_per_group=5, min_detect_pct=0.05))



In [ ]:
df_default = analyzer.run_default(set1={0})

In [ ]:
analyzer.observe_de(df_default,n_rows=30)